# Extract data from pdf and store in table

Quite often, documents in a comapny is stored in pdf format. The strategy of parsing pdf is similar to parsing docx that we first convert the pdf into string and break it down to lists of string. So when we can get the field by searching the field name and take the next line. However, since pdf usually have a less structural format so when we convert them into list of string, it may or may not take the whole row and that randomness will cause extra difficulty to extract the fields. 

## Libraries:

* pdfminer - contains many function and it can be a very powerful tools
* PyPDF2 - more basic library aimed to convert pdf to text
* tabula - reading pdf into table, but it does not always works!

In [1]:
import os
files = os.listdir("pdf")
files = [f for f in files if f.endswith('pdf')]
files = [f for f in files if not f.startswith('.')]
files

['fina_sample_reports.pdf', '3A_e-Premium.pdf', 'credit-report-sample.pdf']

In [10]:
# https://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()
    text = text.split('\n')
    text = [line for line in text if (line is not ' ')]
    tect = [line for line in text if (line is not '')]

    fp.close()
    device.close()
    retstr.close()
    return text

In [12]:
text1 = convert_pdf_to_txt('pdf/fina_sample_reports.pdf')

In [13]:
def extract_paragraph(text, keyword1, keyword2, remove_empty = False):
    paragraph = []
    for line in range(len(text)):
        if any(w in text[line] for w in keyword1):
            paragraph.append(text[line])
            nl = 1
            while any(w in text[line+nl] for w in keyword2) ==  False:
                if (line + nl + 1) >= len(text):
                    paragraph.append(text[line+nl])
                    break
                else:
                    paragraph.append(text[line+nl])
                    nl += 1
            break
    paragraph.append('THE END')
    if remove_empty == True:
        paragraph = [line for line in paragraph if line is not '']
    return paragraph

## Financial report

![](pdf/fina_sample_reports.png)

In [57]:
keyword1 = ['Assets']
keyword2 = ['Financial Analysis CS: Sample Reports']
text1 = extract_paragraph(text1, keyword1, keyword2, remove_empty = True)
text1

['Assets',
 'Cash & Equivalents',
 'Trade Accounts Receivable',
 'Inventory',
 'Other Current Assets',
 'Total Current Assets',
 'Long-Term Investments',
 'Net Fixed Assets',
 'Intangible Assets',
 'Other Non-Current Assets',
 'Total Assets',
 'Liabilities',
 'Accounts Payable',
 'Notes Payable',
 'Accrued Liabilities',
 'Income Taxes Payable',
 'Current Portion of Long-Term Debt',
 'Total Current Liabilities',
 'Long-Term Debt',
 'Other Long-Term Liabilities',
 'Total Long-Term Liabilities',
 'Total Liabilities',
 'Retained Earnings',
 'Total Equity',
 '$336,818',
 '$134,569',
 '$12,985',
 '$98,323',
 '$582,695',
 '$81,197',
 '$412,458',
 '$61,874',
 '$78,390',
 '$1,216,614',
 '$42,787',
 '$88,247',
 '$532,506',
 '$10,014',
 '$111,238',
 '$784,792',
 '$281,809',
 '$55,000',
 '$336,809',
 '$319,978',
 '$127,841',
 '$13,657',
 '$94,325',
 '$555,801',
 '$77,137',
 '$383,750',
 '$58,780',
 '$74,471',
 '$1,149,939',
 '$32,658',
 '$83,835',
 '$530,190',
 '$9,115',
 '$115,676',
 '$771,474',


> The library read the table in columns instead of rows, which is not ideal!

In [30]:
import PyPDF2
pdf_file = open('pdf/fina_sample_reports.pdf')
read_pdf = PyPDF2.PdfFileReader(pdf_file)
number_of_pages = read_pdf.getNumPages()
print number_of_pages
# for i in range(1):
for i in range(10,11):
    page = read_pdf.getPage(i)
#     page_content = page.extractText_c(Tj_sep="\n") + '\n'
    page_content = page.extractText_c().encode('UTF-8') + '\n'
#     page_content = page.extractText_c(Tj_sep="\n").encode('UTF-8') + '\n'
    print '------------ Page %d ------------'%(i+1)
    text_list = page_content.split('\n')
    display(text_list)

74
------------ Page 11 ------------


['Liberty Medical Group',
 'Balance Sheet - Two-Year Comparison',
 'AssetsCash & EquivalentsTrade Accounts Receivable',
 'InventoryOther Current AssetsTotal Current Assets',
 'Long-Term Investments',
 'Net Fixed Assets',
 'Intangible AssetsOther Non-Current AssetsTotal Assets',
 'LiabilitiesAccounts Payable',
 'Notes Payable',
 'Accrued LiabilitiesIncome Taxes Payable',
 'Current Portion of Long-Term Debt',
 'Total Current LiabilitiesLong-Term Debt',
 'Other Long-Term Liabilities',
 'Total Long-Term Liabilities',
 'Total LiabilitiesRetained EarningsTotal Equity',
 'Total Liabilities and Equity2008$336,818$134,569$12,985$98,323$582,695$81,197$412,458$61,874$78,390$1,216,614$42,787$88,247$532,506$10,014$111,238$784,792$281,809$55,000$336,809$1,121,601$95,013$95,013$1,216,6142007$319,978$127,841$13,657$94,325$555,801$77,137$383,750$58,780$74,471$1,149,939$32,658$83,835$530,190$9,115$115,676$771,474$263,352$36,000$299,352$1,070,826$79,113$79,113$1,149,939$ Variance$16,840$6,728-$672$3,998$

> The original PyPDF2 will try to read the file in rows, however, it does not work well in this case.

## Modified PyPDF2

In [42]:
for i in range(10,11):
    page = read_pdf.getPage(i)
#     page_content = page.extractText_c(Tj_sep="\n") + '\n'
#     page_content = page.extractText_c().encode('UTF-8') + '\n'
    page_content = page.extractText_c(Tj_sep="\n").encode('UTF-8') + '\n'
    print '------------ Page %d ------------'%(i+1)
    text_list = page_content.split('\n')
    display(text_list[:50])

------------ Page 11 ------------


['Liberty Medical Group',
 'Balance Sheet - Two-Year Comparison',
 '',
 'Assets',
 'Cash & EquivalentsTrade Accounts Receivable',
 '',
 'Inventory',
 'Other Current AssetsTotal Current Assets',
 'Long-Term Investments',
 'Net Fixed Assets',
 '',
 'Intangible Assets',
 'Other Non-Current AssetsTotal Assets',
 '',
 'LiabilitiesAccounts Payable',
 'Notes Payable',
 '',
 'Accrued LiabilitiesIncome Taxes Payable',
 'Current Portion of Long-Term Debt',
 '',
 'Total Current LiabilitiesLong-Term Debt',
 'Other Long-Term Liabilities',
 'Total Long-Term Liabilities',
 '',
 'Total Liabilities',
 'Retained EarningsTotal Equity',
 '',
 'Total Liabilities and Equity',
 '2008',
 '$336,818',
 '$134,569',
 '$12,985',
 '$98,323',
 '$582,695',
 '$81,197',
 '$412,458',
 '$61,874',
 '$78,390',
 '$1,216,614',
 '$42,787',
 '$88,247',
 '$532,506',
 '$10,014',
 '$111,238',
 '$784,792',
 '$281,809',
 '$55,000',
 '$336,809',
 '$1,121,601',
 '$95,013']

> The modified PyPDF2 allow you to add a break between strings, however, the result of this parsing is still poor.

## Try using tabula to extract the data in table

In [59]:
import tabula

# Read pdf into DataFrame
df = tabula.read_pdf('pdf/fina_sample_reports.pdf', pages=11)
df

,Cash & Equivalents,"$336,818","$319,978","$16,840",5.3%
0,Trade Accounts Receivable,"$134,569","$127,841","$6,728",5.3%
1,Inventory,"$12,985","$13,657",-$672,-4.9%
2,Other Current Assets,"$98,323","$94,325","$3,998",4.2%
3,Total Current Assets,"$582,695","$555,801","$26,894",4.8%
4,Long-Term Investments,"$81,197","$77,137","$4,060",5.3%
5,Net Fixed Assets,"$412,458","$383,750","$28,708",7.5%
6,Intangible Assets,"$61,874","$58,780","$3,094",5.3%
7,Other Non-Current Assets,"$78,390","$74,471","$3,919",5.3%
8,Total Assets,"$1,216,614","$1,149,939","$66,675",5.8%
9,Liabilities,NaN,NaN,NaN,NaN


> tabula can output a table but it still quite different from the original table in the report. 

## Modified DataFrame

In [60]:
import pandas as pd
df.T.reset_index().T.reset_index(drop = True)

,0,1,2,3,4
0,Cash & Equivalents,"$336,818","$319,978","$16,840",5.3%
1,Trade Accounts Receivable,"$134,569","$127,841","$6,728",5.3%
2,Inventory,"$12,985","$13,657",-$672,-4.9%
3,Other Current Assets,"$98,323","$94,325","$3,998",4.2%
4,Total Current Assets,"$582,695","$555,801","$26,894",4.8%
5,Long-Term Investments,"$81,197","$77,137","$4,060",5.3%
6,Net Fixed Assets,"$412,458","$383,750","$28,708",7.5%
7,Intangible Assets,"$61,874","$58,780","$3,094",5.3%
8,Other Non-Current Assets,"$78,390","$74,471","$3,919",5.3%
9,Total Assets,"$1,216,614","$1,149,939","$66,675",5.8%


In [ ]:
text2 = convert_pdf_to_txt('pdf/fina_sample_reports.pdf')